__Scripts for analysing genre fields in our dataset.__

@Andreas Lüschow

12.11.2020

## Imports

In [ ]:
from IPython.display import display

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

pd.options.display.max_columns = None

## Constants

In [ ]:
# Input
INPUT_CSV = "./data/input/all_genre_data.csv"
GENRE_FILE = "./data/input/genres.txt"
FIELD_FILE = "./data/input/fields.txt"

# Output
INFO_DF_CSV = "./data/output/field_info_df.csv"
GENRE_INFO_DF_CSV = "./data/output/genre_info_df.csv"

## Load Data
* source DataFrame
* list of genres
* list of fields with genre information

In [ ]:
src_df = pd.read_csv(INPUT_CSV, sep="\t", index_col='Unnamed: 0', low_memory=False)
src_df.shape

In [ ]:
src_df[:5]

In [ ]:
genres = [line.strip() for line in open(GENRE_FILE)]
assert len(genres) == 1319
print(genres[:20])

In [ ]:
fields = [line.strip() for line in open(FIELD_FILE)]
print(fields)

## Analyse single fields in source data

### Create dictionary with field attributes
Each row/key in the dictionary is the name of a genre-relevant field/subfield in the data.
* __notna_sum__: absolute number of records that have a value in field X (not necessarily a genre!)
* __notna_rel__: relative of records that have a value in field X ("1.0" would mean: all records have this field)
* __genres__: set of genres that can be found in this field
* __genre_count__: number of genres in this field
* for each field, each genre has a value with its absolute occurrence in the data (__[genre_name]__)
* for each genre there is also a column for its relative appearance in the field (__[genre name] + _rel__)

In [ ]:
info_dict = {}

In [ ]:
for f in fields:
    info_dict[f] = {}
    info_dict[f]["notna_sum"] = src_df[f].notna().sum()
    info_dict[f]["notna_rel"] = (src_df[f].notna().sum()/len(src_df.index)).round(4)
    
    # add genres
    field_genres = []
    count = 0
    for elem in src_df[f].str.split(";"):
        if isinstance(elem, list):
            for value in elem:
                genre = value.strip()
                if genre in genres:
                    if genre in info_dict[f].keys():
                        info_dict[f][genre] += 1
                    else:
                        info_dict[f][genre] = 1
                    field_genres.append(genre)
        elif str(elem).strip() in genres:
            genre = str(elem).strip()
            if genre in info_dict.keys():
                info_dict[f][genre] += 1
            else:
                info_dict[f][genre] = 1
            field_genres.append(genre)
    info_dict[f]["genres"] = set(field_genres)
    info_dict[f]["genre_count"] = len(info_df["genres"].loc[f])
    
    for genre in genres:
        if genre in info_dict[f].keys():
            info_dict[f][genre + "_rel"] = (info_dict[f][genre]/info_dict[f]["notna_sum"]).round(6)

### Convert to DataFrame

In [ ]:
info_df = pd.DataFrame.from_dict(info_dict, orient="index")
info_df.to_csv(INFO_DF_CSV, sep="\t")
info_df
info_df.shape

### Create dictionary based on genres

In [ ]:
# create dataframe (not_nan_df) that contains only not-nan-columns for each field
_tmp_df = info_df.copy()
not_nan_df = (_tmp_df.stack()
   .reset_index(level=1)
   .groupby(level=0, sort=False)
   ['level_1'].apply(list)
)

The genre info dictionary contains the following information:
* __max_occurrence__: maximum occurrence of this genre in a field (_absolute_ number!)
* __max_field__: field in which this genre appears mostly (based on _absolute_ (!) numbers)
* __in_fields__: list of fields where this genre appears
* __not_in_fields__: list of fields where this genre does not appear
* __rel_X__: relative occurrence in field X

In [ ]:
genre_info_dict = {}

In [ ]:
for g in genres:
    in_fields = []
    not_fields = []
    if g in info_df.keys():
        max_value = info_df[g].max()
        max_field = info_df[info_df[g] == info_df[g].max()].index[0]
        for f in fields:
            if g in not_nan_df[f]:
                in_fields.append(f)
            else:
                not_fields.append(f)
    
        genre_info_dict[g] = {}
        genre_info_dict[g]["max_occurrence"] = max_value
        genre_info_dict[g]["max_field"] = max_field
        genre_info_dict[g]["in_fields"] = in_fields
        genre_info_dict[g]["not_in_fields"] = not_fields
        for f in fields:
            genre_info_dict[g]["rel_" + f] = info_df[g + "_rel"].loc[f]

### Convert to DataFrame

In [ ]:
genre_info_df = pd.DataFrame.from_dict(genre_info_dict, orient="index")
genre_info_df.to_csv(GENRE_INFO_DF_CSV, sep="\t")
genre_info_df
genre_info_df.shape

In [ ]:
# look into example field
genre_info_df[genre_info_df["max_field"] == "013D a"]

Based on the data of the genre dictionary we can calculate for each field which genre appears the most (__max_field_rel_genre__) and to which relative extent (__max_field_rel__) and add this information to the DataFrame that is based on fields (see above).

In [ ]:
max_field_rel_list = []
max_field_rel_genre_list = []

for f in fields:
    max_field_rel = genre_info_df["rel_" + f].max()
    max_field_rel_genre = genre_info_df[genre_info_df["rel_" + f] == genre_info_df["rel_" + f].max()].index[0]
    max_field_rel_list.append(max_field_rel)
    max_field_rel_genre_list.append(max_field_rel_genre)
    
info_df["max_field_rel"] = max_field_rel_list
info_df["max_field_rel_genre"] = max_field_rel_genre_list

In [ ]:
info_df[["max_field_rel", "max_field_rel_genre"]]

In [ ]:
info_df.to_csv(INFO_DF_CSV, sep="\t")